In [1]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1710535308268_0008,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sc

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-1>

### Estaciones que han realizado mediciones de NO y CO número de mediciones y rango de fechas, consultando desde tablas externas creadas en HIVE:

In [3]:
df_no_co=spark.sql("SELECT 'co' indicador,\
                        nombre,\
                        nombrecorto,\
                        count(1) num_mediciones,\
                        min(fecha) fecha_inicio,\
                        max(fecha) fecha_fin \
                FROM proyecto1.co_trusted \
                group by indicador,nombre,nombrecorto\
                union ALL\
                SELECT 'no' indicador, \
                        nombre,\
                        nombrecorto,\
                        count(1) num_mediciones,\
                        min(fecha) fecha_inicio,\
                        max(fecha) fecha_fin \
                        FROM proyecto1.no_trusted \
                group by indicador,nombre,nombrecorto")
df_no_co.show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+------------------------------------------------------------------------+-----------+--------------+-------------------+-------------------+
|indicador|nombre                                                                  |nombrecorto|num_mediciones|fecha_inicio       |fecha_fin          |
+---------+------------------------------------------------------------------------+-----------+--------------+-------------------+-------------------+
|co       |Politecnico Colombiano Jaime Isaza Cadavid - Medellin                   |MED-PJIC   |8761          |2021-12-14 04:00:00|2022-12-14 04:00:00|
|co       |_OFF-Estación Tráfico Sur                                               |SUR-TRAF   |8761          |2021-12-14 04:00:00|2022-12-14 04:00:00|
|co       |_OFF-Girardota - S.O.S Aburrá Norte                                     |GIR-SOSN   |8761          |2021-12-14 04:00:00|2022-12-14 04:00:00|
|no       |Institucion Universitaria ITM Robledo                                   |MED-

### Estaciones que han realizado mediciones de CO2 y SO2, número de mediciones y rango de fechas, consultado tablas externas creadas desde Redshift Spectrum:

In [4]:
df_co2_so2=spark.sql("SELECT 'so2' indicador,\
                        nombre,\
                        nombrecorto,\
                        count(1) num_mediciones,\
                        min(fecha) fecha_inicio,\
                        max(fecha) fecha_fin \
                FROM proyecto1_spectrumdb.so2_trusted_spectrum \
                group by indicador,nombre,nombrecorto\
                union ALL\
                SELECT 'no2' indicador, \
                        nombre,\
                        nombrecorto,\
                        count(1) num_mediciones,\
                        min(fecha) fecha_inicio,\
                        max(fecha) fecha_fin \
                        FROM proyecto1_spectrumdb.no2_trusted_spectrum \
                group by indicador,nombre,nombrecorto")
df_co2_so2.show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+------------------------------------------------------------------------+-----------+--------------+-------------------+-------------------+
|indicador|nombre                                                                  |nombrecorto|num_mediciones|fecha_inicio       |fecha_fin          |
+---------+------------------------------------------------------------------------+-----------+--------------+-------------------+-------------------+
|so2      |_OFF-Girardota - S.O.S Aburrá Norte                                     |GIR-SOSN   |8761          |2021-12-14 04:00:00|2022-12-14 04:00:00|
|no2      |Institucion Universitaria ITM Robledo                                   |MED-ITMR   |8761          |2021-12-14 04:00:00|2022-12-14 04:00:00|
|no2      |Itagüí - Casa de Justicia Itagüí                                        |ITA-CJUS   |8761          |2021-12-14 04:00:00|2022-12-14 04:00:00|
|no2      |Politecnico Colombiano Jaime Isaza Cadavid - Medellin                   |MED-

### Edad Promedio por Municipio, y cuál es el municipio con mayor edad promedio:

In [5]:
df_pob=spark.sql('select nombremunicipio,\
       sum(hombres_cabecera+hombres_centropoblado+hombresruraldisperso) total_hombres,\
       sum(mujeres_cabecera+mujeres_centropoblado+mujeresruraldisperso) total_mujeres,\
       sum(edad*(hombres_cabecera+hombres_centropoblado+hombresruraldisperso+mujeres_cabecera+mujeres_centropoblado+mujeresruraldisperso))/\
       sum(hombres_cabecera+hombres_centropoblado+hombresruraldisperso+mujeres_cabecera+mujeres_centropoblado+mujeresruraldisperso) edad_promedio\
    from proyecto1_spectrumdb.poblacionantioquia_spectrum\
    where edad is not null\
    group by nombremunicipio')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
df_pob.sort("edad_promedio", ascending=[False]).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+-------------+-------------+------------------+
|nombremunicipio|total_hombres|total_mujeres|     edad_promedio|
+---------------+-------------+-------------+------------------+
|        Armenia|       1912.0|       1907.0|40.450641529196126|
|       Envigado|      97955.0|     114482.0|  39.9075396470483|
|       Carolina|       1646.0|       1697.0| 39.82829793598564|
|        Ebéjico|       5285.0|       5053.0| 39.51808860514606|
|       Titiribí|       4118.0|       4198.0| 39.31036556036556|
|         Jericó|       5580.0|       5677.0|39.085724438127386|
|       Fredonia|       9415.0|       9375.0| 38.77823310271421|
|        Támesis|       7569.0|       7443.0|38.737276845190515|
|      Heliconia|       2464.0|       2245.0| 38.68464642174559|
|        Venecia|       5063.0|       5217.0|38.665856031128406|
|     Montebello|       3007.0|       2874.0|38.400102023465394|
|     Valparaíso|       2758.0|       2673.0|38.373780150985084|
|  Santa Bárbara|      10

### Municipio con Menor Edad Promedio

In [7]:
df_pob.sort("edad_promedio", ascending=[True]).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+-------------+-------------+------------------+
|  nombremunicipio|total_hombres|total_mujeres|     edad_promedio|
+-----------------+-------------+-------------+------------------+
|          Murindó|       2362.0|       2280.0|23.291684618698838|
| Vigía Del Fuerte|       4049.0|       4090.0|25.968546504484582|
|           Mutatá|       6403.0|       6204.0| 27.46180693265646|
|            Turbo|      55316.0|      59118.0|27.671417585682576|
|         El Bagre|      23851.0|      23791.0|  27.7971117921162|
|         Zaragoza|      12099.0|      11968.0| 27.92230024514896|
|           Carepa|      22989.0|      22947.0|28.004636886102404|
|San Juan De Urabá|       9904.0|      10088.0|28.090186074429774|
|          Necoclí|      19094.0|      19326.0|28.104919312857888|
|          Cáceres|      13794.0|      12666.0|28.193461829176115|
|            Nechí|      12369.0|      11697.0| 28.22409208011302|
|           Tarazá|      11272.0|      11554.0|28.269473407517

### Edad Promedio del Departamento de Antioquia

In [8]:
df_pob.groupby().avg('edad_promedio').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+
|avg(edad_promedio)|
+------------------+
| 33.73590774624305|
+------------------+

### Municipio con mayor indice de NO2 y cuanta es su poblacion y edad promedio:

In [13]:
sql_no2_pob ="with poblacion as ( \
    select nombremunicipio municipio, \
       sum(hombres_cabecera+hombres_centropoblado+hombresruraldisperso+mujeres_cabecera+mujeres_centropoblado+mujeresruraldisperso) poblacion, \
       sum(edad*(hombres_cabecera+hombres_centropoblado+hombresruraldisperso+mujeres_cabecera+mujeres_centropoblado+mujeresruraldisperso))/ \
    sum(hombres_cabecera+hombres_centropoblado+hombresruraldisperso+mujeres_cabecera+mujeres_centropoblado+mujeresruraldisperso) edad_promedio \
    from proyecto1_spectrumdb.poblacionantioquia_spectrum \
    where edad is not null \
    group by nombremunicipio \
), calidad_no2 as ( \
select municipio,avg(calidad) calidad_no2 \
FROM proyecto1_spectrumdb.no2_trusted_spectrum a \
inner join proyecto1.ubicaciones_mediciones b on a.nombrecorto=b.ubicacion \
group by municipio \
) \
select a.municipio,a.calidad_no2,b.poblacion,b.edad_promedio \
from calidad_no2 A \
LEFT JOIN poblacion B ON A.municipio=B.municipio"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
df_no2_pob=spark.sql(sql_no2_pob)
df_no2_pob.sort("calidad_no2", ascending=[False]).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+------------------+---------+------------------+
|municipio|       calidad_no2|poblacion|     edad_promedio|
+---------+------------------+---------+------------------+
|Girardota|   4.3084887569914|  49045.0| 35.87234172698542|
| Medellín|3.2056439523647136|2347656.0|36.150517367110005|
| Sabaneta| 1.239796826846263|  81797.0|37.185728082937025|
+---------+------------------+---------+------------------+

### Municipio con menor indice de CO y cuantos de su población son hombres y cuantos mujeres

In [15]:
sql_co_pob ="with poblacion as ( \
    select nombremunicipio municipio, \
       sum(hombres_cabecera+hombres_centropoblado+hombresruraldisperso+mujeres_cabecera+mujeres_centropoblado+mujeresruraldisperso) poblacion, \
       sum(edad*(hombres_cabecera+hombres_centropoblado+hombresruraldisperso+mujeres_cabecera+mujeres_centropoblado+mujeresruraldisperso))/ \
    sum(hombres_cabecera+hombres_centropoblado+hombresruraldisperso+mujeres_cabecera+mujeres_centropoblado+mujeresruraldisperso) edad_promedio \
    from proyecto1_spectrumdb.poblacionantioquia_spectrum \
    where edad is not null \
    group by nombremunicipio \
) , calidad_co as ( \
select municipio,avg(calidad) calidad_co \
FROM proyecto1.co_trusted a \
inner join proyecto1.ubicaciones_mediciones b on a.nombrecorto=b.ubicacion \
group by municipio \
) \
select a.municipio,a.calidad_co,b.poblacion,b.edad_promedio \
from calidad_co A \
LEFT JOIN poblacion B ON A.municipio=B.municipio"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
df_co_pob=spark.sql(sql_co_pob)
df_co_pob.sort("calidad_co", ascending=[True]).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+------------------+---------+------------------+
|municipio|        calidad_co|poblacion|     edad_promedio|
+---------+------------------+---------+------------------+
| Medellín|1.3884442415249654|2347656.0|36.150517367110005|
| Sabaneta|               4.0|  81797.0|37.185728082937025|
|Girardota| 4.310000000000188|  49045.0| 35.87234172698542|
+---------+------------------+---------+------------------+